In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Load datasets
demand_data = pd.read_csv("D:/Ullas/MSc Data Analytics Capstone Project/Datasets/Passenger Journeys by Public Transport.csv")
demographic_data = pd.read_csv("D:/Ullas/MSc Data Analytics Capstone Project/Datasets/Demographic.csv")
weather_data = pd.read_csv("D:/Ullas/MSc Data Analytics Capstone Project/Datasets/weather.csv")

In [ ]:
demand_data.head()

In [ ]:
demographic_data.head()

In [ ]:
weather_data.head()

In [ ]:
pd.set_option('display.max_rows', None)
print(demand_data)

In [ ]:
# Displaying basic information about the datasets
print("Demand Data Info:")
print(demand_data.info())

print("\nDemographic Data Info:")
print(demographic_data.info())

print("\nWeather Data Info:")
print(weather_data.info())

# Summary statistics
print("\nDemand Data Summary Statistics:")
print(demand_data.describe())

print("\nDemographic Data Summary Statistics:")
print(demographic_data.describe())

print("\nWeather Data Summary Statistics:")
print(weather_data.describe())

In [ ]:
demand_data.columns

In [ ]:
demographic_data.columns

In [ ]:
weather_data.columns

# Preparing the first dataset: demand_data

In [ ]:
# Renaming columns
demand_data.rename(columns={'Weeks of the year': 'Weeks', 'VALUE': 'Demand'}, inplace=True)

In [ ]:
demand_data.drop(['STATISTIC','TLIST(A1)','UNIT'], axis = 1, inplace = True)

In [ ]:
#converting year and weeks columns to date column 
demand_data['Date'] = demand_data['Year'].astype(str) + '-W' + demand_data['Weeks'].str.slice(start=5).astype(int).apply(lambda x: f'{x:02}')
demand_data['Date'] = pd.to_datetime(demand_data['Date'] + '-1', format='%Y-W%U-%w')
demand_data['Date'] = demand_data['Date'].dt.strftime('%d/%m/%Y')

In [ ]:
desired_order = ['Year', 'Weeks', 'Date', 'Mode of Transport', 'Statistic Label', 'Demand', 'C03935V04687', 'C01198V01436']

demand_data = demand_data.reindex(columns=desired_order)

#Dealing with the missing values on the dataset
demand_data.interpolate(method='linear', inplace=True)

#formating the Deamnd column to make it more simpler 
demand_data['Demand'] = demand_data['Demand'].apply(lambda x: "{:,.0f}".format(x))

# remove commas on Demand
demand_data['Demand'] = demand_data['Demand'].str.replace(',', '').astype(int)

In [ ]:
demand_data.drop(['C03935V04687','C01198V01436'], axis = 1, inplace = True)

In [ ]:
demand_data.columns

In [ ]:
demand_data.head()

In [ ]:
demand_data['Date'] = pd.to_datetime(demand_data['Date'], format='%d/%m/%Y')

In [ ]:
demand_data['Date'] = pd.to_datetime(demand_data['Date'])

In [ ]:
unique_transport_modes = demand_data["Mode of Transport"].unique()

In [ ]:
print(unique_transport_modes)

In [ ]:
# Mapping the names in the column 
transport_mapping = {
    "Dublin Metro Bus": "Dart", "Bus, excluding Dublin Metro": "Dublin Bus", "All public transport, excluding LUAS" : "Luas" 
}

In [ ]:
# Replace the old names with new names
demand_data["Mode of Transport"] = demand_data["Mode of Transport"].replace(transport_mapping)

In [ ]:
demand_data = demand_data[demand_data['Date'] <= '2023-08-28']

In [ ]:
pd.set_option('display.max_rows', None)
print(demand_data)

In [ ]:
total_lines = len(demand_data)
print("Total lines of data:", total_lines)

# Preparing the Second dataset: demographic_data

In [ ]:
demographic_data.head()

In [ ]:
total_lines = len(demographic_data)
print("Total lines of data:", total_lines)

In [ ]:
unique_Statistic_label = demographic_data["Statistic Label"].unique()

In [ ]:
print(unique_Statistic_label)

In [ ]:
# Renaming columns
demographic_data.rename(columns={'Sex': 'Gender'}, inplace=True)

In [ ]:
demographic_data.drop(['STATISTIC','TLIST(A1)','C02199V02655','C02076V02508','UNIT','Statistic Label','VALUE'], axis = 1, inplace = True)

In [ ]:
demographic_data.head()

In [ ]:
# Forward fill and Backward fill
demographic_data.fillna(method='ffill', inplace=True)  
demographic_data.fillna(method='bfill', inplace=True) 

In [ ]:
demographic_data.interpolate(method='linear', inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
print(demographic_data)

In [ ]:
demographic_data.columns

In [ ]:
total_lines = len(demographic_data)
print("Total lines of data:", total_lines)

# Preparing the Third dataset: weather_data

In [ ]:
weather_data.head()

In [ ]:
# Renaming columns
weather_data.rename(columns={'ind': 'Indicator', 'rain': 'Rain(mm)','maxt':'Temperature(c)','soil':'Soil'}, inplace=True)

In [ ]:
weather_data.drop(['ind.1','ind.2','mint','gmin','Soil','Indicator','Temperature(c)'], axis = 1, inplace = True)

In [ ]:
weather_data.columns

In [ ]:
# Converting date column to datetime format
weather_data['date'] = pd.to_datetime(weather_data['date'], format='%d-%b-%y')

In [ ]:
start_date = pd.to_datetime('1941-01-01')
end_date = pd.to_datetime('2023-05-31')

In [ ]:
weather_data = weather_data[(weather_data['date'] >= start_date) & (weather_data['date'] <= end_date)]

In [ ]:
total_lines = len(weather_data)
print("Total lines of data:", total_lines)

In [ ]:
for column in ["Rain(mm)"]:
    weather_data[column] = weather_data.groupby('date')[column].transform(
        lambda x: x.fillna(np.random.choice(x.dropna()))
    )

In [ ]:
pd.set_option('display.max_rows', None)
print(weather_data)

In [ ]:
# Set the date column as the DataFrame index
weather_data.set_index('date', inplace=True)

# Resample the data to a weekly frequency and calculate the sum of 'Rain(mm)' for each week
weekly_weather_data = weather_data.resample('W').sum()

# Reset the index to make 'date' a column again
weekly_weather_data.reset_index(inplace=True)

# Print the resulting DataFrame
print(weekly_weather_data)

In [ ]:
total_lines = len(weekly_weather_data)
print("Total lines of data:", total_lines)

In [ ]:
weather_data.columns

In [ ]:
start_date = '2019-01-06'
end_date = '2023-06-04'

In [ ]:
filtered_weather_data = weekly_weather_data[(weekly_weather_data['date'] >= start_date) & (weekly_weather_data['date'] <= end_date)]

In [ ]:
pd.set_option('display.max_rows', None)
print(filtered_weather_data)

In [ ]:
total_lines = len(filtered_weather_data)
print("Total lines of data:", total_lines)

In [ ]:
demand_data['Date'] = pd.to_datetime(demand_data['Date'])
filtered_weather_data['date'] = pd.to_datetime(filtered_weather_data['date'])
#Creating a merged Dataframe 
merged_data = pd.merge(demand_data, filtered_weather_data, left_on='Date', right_on='date', how='left')
merged_data.drop(columns=['date'], inplace=True)
# Creating a copy of filtered_weather_data to perform resampling
filtered_weather_data_copy = filtered_weather_data.copy()
filtered_weather_data_copy.set_index('date', inplace=True)
# Resampling the Rain column in the copied weather dataset to weekly and calculated the mean
filtered_weather_data_copy['Rain(mm)'] = filtered_weather_data_copy['Rain(mm)'].resample('W').mean()
# Fill missing values in the Rain(mm) column with the corresponding weekly mean
merged_data['Rain(mm)'] = merged_data['Date'].apply(
    lambda x: filtered_weather_data_copy.loc[filtered_weather_data_copy.index.week == x.isocalendar().week, 'Rain(mm)'].values[0]
    if not pd.isna(x) else x
)

In [ ]:
pd.set_option('display.max_rows', None)
print(merged_data)

In [ ]:
merged_data.drop(columns=['Statistic Label'], inplace=True)

In [ ]:
total_lines = len(merged_data)
print("Total lines of data:", total_lines)

In [ ]:
pd.set_option('display.max_rows', None)
print(merged_data)